In [2]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
from datasets import ClickDataset
from sklearn.preprocessing import MinMaxScaler
from utils import create_dtw_pairs
from models import SiameseNetwork, ContrastiveLoss, train_siamese, evaluate_siamese

dataset = ClickDataset('Click_details', transform=MinMaxScaler())
print("Dataset created")
pairs1, pairs2, labels = create_dtw_pairs(dataset, 6000)
print("Pairs created")

pairs1 = torch.tensor(pairs1, dtype=torch.float32)
pairs2 = torch.tensor(pairs2, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

model = SiameseNetwork()
criterion = ContrastiveLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

pairs1_train, pairs1_val, pairs2_train, pairs2_val, labels_train, labels_val = train_test_split(pairs1, pairs2, labels, test_size=0.2, random_state=42)

train_dataset = TensorDataset(pairs1_train,
                             pairs2_train,
                             labels_train)

val_dataset = TensorDataset(pairs1_val,
                           pairs2_val,
                           labels_val)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

train_siamese(model, train_loader, criterion, optimizer, epochs=5)

evaluate_siamese(model, val_loader)

NameError: name 'ClickDataset' is not defined